In [52]:
import tensorflow as tf
import numpy as np
import keras
from keras import models, layers
from keras.models import load_model, Model, Sequential
from keras.layers import Reshape, Merge,Activation, Conv2D, Flatten,Input, MaxPooling2D, BatchNormalization, Flatten, Dense, Lambda
from keras.layers.advanced_activations import LeakyReLU
from keras.layers.merge import concatenate
from keras.layers.core import Activation, Reshape

### Introdução a YOLO ###

[Redes Neurais Convolucionais](http://cs231n.github.io/convolutional-networks/) (CNNs) tem sido utilizadas em diversas aplicações, sendo bastante populares para aplicações voltadas a imagens e atingindo um alto índice de acurácia. Entretanto, em algumas situações, é necessário reconhecer objetos na imagem e localizá-los com uma performance quase ou melhor que a humana.

Existem algumas abordagens que utilizam CNNs também para detecção, como [R-CNNs](https://arxiv.org/abs/1506.01497) que são consideravelmente custosas e um pouco lentas devido a utilização de diferentes pipelines para classificação e geração de bouding boxes. A YOLO realiza essas etapas simultaneamente, retornando uma melhor performance em tempo real. Atualmente, existem duas versões da YOLO, onde a primeira diferença entre elas é a arquitetura da Rede Neural utilizada.


[Arquitetura YOLOv1](https://pjreddie.com/media/files/papers/yolo.pdf)

No primeiro modelo da YOLO, a rede neural criada foi inspirada na [GoogleLeNet](https://www.cs.unc.edu/~wliu/papers/GoogLeNet.pdf), rede neural utilizada para classificação de imagens com menor taxa de erro no desafio ImageNet de 2014 (6.7% de erro).

Neste modelo, existem 24 camadas convolucionais para extração de características, enquanto as duas últimas camadas da rede são completamente conectadas para identificar o objeto e suas coordenadas, conforme Figura abaixo.

![alt text](../imgs/yolo_architecture.png "Title")

Na figura, percebe-se que as camadas convolucionais são divididas em 6 blocos principais, sendo os dois primeiros formados apenas por uma camada convolucional e max-pooling para extração das características primárias. Apesar do uso da técnica max-pooling reduzir consideravelmente a dimensão da imagem (widthxheight), a profundidade, obtida pela quantidade de filtros aplicados a cada camada, permanece constante. 

Dessa forma, para arquiteturas complexas como a YOLO, a utilização de muitas camadas convolucionais com um número de filtros elevado resulta em um custo computacional muito alto. Uma forma de evitar isso e reduzir a largura da saída de cada camada é aplicar camadas convolutivas de dimensões 1x1xn, um conceito abordado em [Network in Network](https://arxiv.org/abs/1312.4400). Ao utilizar essas dimensões, o número de entradas (neurônios) da próxima camada reduz em um fator dado pela equação abaixo:

$$\frac{quantidade-de-filtros-da-camada-atual}{quantidade-de-filtros-da-camada-convolutiva-1x1}$$

Assim, a partir do terceiro bloco da rede, uma camada convolucional 1x1 é aplicada antes da aplicação de uma camada 3x3. Além disso, a quantidade de filtros dobra a cada camada, iniciando com 124 filtros e encerrando com 1024. Esse aumento de filtros foi inspirado no modelo de rede [VGG](https://arxiv.org/abs/1409.1556), tida como uma das redes mais utilizadas como base de detecção.

Esse processo repete-se até a vigésima camada, uma vez que essas são as camadas responsáveis pela extração de características e reconhecimento da imagem. O modelo é então pré-treinado utilizando estas camadas, para, em seguida, ser utilizado para detecção. Para isso, as últimas duas camadas convolucionais foram adicionadas, assim como as camadas completamente conectadas.

Por fim, para realizar detecção utilizando essa arquitetura, quatro camadas convolucionais e duas camadas completamente conectadas são adicionadas. Entretanto, o artigo da YOLO não descreve os hiper-parâmetros destas camadas, de forma que a implementação da yolo neste notebook não utilizou as mesmas.


In [47]:
# the function to implement the orgnization layer (thanks to github.com/allanzelener/YAD2K)
def space_to_depth_x2(x):
    return tf.space_to_depth(x, block_size=2)

In [53]:
def yolo(input_image):
    """
    Implementação da arquitetura da YOLO dividida nos seis blocos convolucionais e nas camadas
    completamente conectadas.
    Parâmetro: dimensão da imagem
    Retorna: modelo final implementado
    """    
    
    model = Sequential()
    
    # Bloco 1 - Uma camada convolucional não muito densa (64 filtros)
    model.add(Conv2D(64, (7,7), strides=(2,2), padding='same', name='conv_1',input_shape=(416, 416, 3)))
    model.add(LeakyReLU(alpha=0.1))
    model.add(MaxPooling2D(pool_size=(2, 2)))

    # Bloco 2 
    model.add(Conv2D(192, (3,3), strides=(2,2), padding='same', name='conv_2'))
    model.add(LeakyReLU(alpha=0.1))
    model.add(MaxPooling2D(pool_size=(2, 2)))
    
    #Bloco 3 - Transição entre dimensões 256 e 512 a partir da utilização de camadas
    #convolucionais 1x1
    model.add(Conv2D(128, (1,1), strides=(1,1), padding='same', name='conv_3'))
    model.add(LeakyReLU(alpha=0.1))
    model.add(Conv2D(256, (3,3), strides=(1,1), padding='same', name='conv_4'))
    model.add(LeakyReLU(alpha=0.1))
    model.add(Conv2D(256, (1,1), strides=(1,1), padding='same', name='conv_5'))
    model.add(LeakyReLU(alpha=0.1))
    model.add(Conv2D(512, (3,3), strides=(1,1), padding='same', name='conv_6'))
    model.add(LeakyReLU(alpha=0.1))
    model.add(MaxPooling2D(pool_size=(2, 2)))
    
    #Bloco 4  - Transição entre dimensões 512 e 1024 a partir de camadas convolucionais
    # 1x1
    model.add(Conv2D(256, (1,1), strides=(1,1), padding='same', name='conv_7'))
    model.add(LeakyReLU(alpha=0.1))
    model.add(Conv2D(512, (3,3), strides=(1,1), padding='same', name='conv_8'))
    model.add(LeakyReLU(alpha=0.1))
    model.add(Conv2D(256, (1,1), strides=(1,1), padding='same', name='conv_9'))
    model.add(LeakyReLU(alpha=0.1))
    model.add(Conv2D(512, (3,3), strides=(1,1), padding='same', name='conv_10'))
    model.add(LeakyReLU(alpha=0.1))
    model.add(Conv2D(256, (1,1), strides=(1,1), padding='same', name='conv_11'))
    model.add(LeakyReLU(alpha=0.1))
    model.add(Conv2D(512, (3,3), strides=(1,1), padding='same', name='conv_12'))
    model.add(LeakyReLU(alpha=0.1))
    model.add(Conv2D(256, (1,1), strides=(1,1), padding='same', name='conv_13'))
    model.add(LeakyReLU(alpha=0.1))
    model.add(Conv2D(512, (3,3), strides=(1,1), padding='same', name='conv_14'))
    model.add(LeakyReLU(alpha=0.1))
    model.add(Conv2D(512, (1,1), strides=(1,1), padding='same', name='conv_15'))
    model.add(LeakyReLU(alpha=0.1))
    model.add(Conv2D(1024, (3,3), strides=(1,1), padding='same', name='conv_16'))
    model.add(LeakyReLU(alpha=0.1))
    model.add(MaxPooling2D(pool_size=(2, 2)))    
    
    #Bloco 5 - Transição entre dimensões 512 e 1024 a partir de camadas convolucionais
    # 1x1
    model.add(Conv2D(512, (1,1), strides=(1,1), padding='same', name='conv_17'))
    model.add(LeakyReLU(alpha=0.1))
    model.add(Conv2D(1024, (3,3), strides=(1,1), padding='same', name='conv_18'))
    model.add(LeakyReLU(alpha=0.1))
    model.add(Conv2D(512, (1,1), strides=(1,1), padding='same', name='conv_19'))
    model.add(LeakyReLU(alpha=0.1))
    model.add(Conv2D(1024, (3,3), strides=(1,1), padding='same', name='conv_20'))
    model.add(LeakyReLU(alpha=0.1))  
    model.add(Conv2D(1024, (3,3), strides=(1,1), padding='same', name='conv_21'))
    model.add(LeakyReLU(alpha=0.1)) 
    model.add(Conv2D(1024, (3,3), strides=(1,1), padding='same', name='conv_22'))
    model.add(LeakyReLU(alpha=0.1)) 
    
    #Bloco 6
    model.add(Conv2D(1024, (3,3), strides=(1,1), padding='same', name='conv_23'))
    model.add(LeakyReLU(alpha=0.1))
    model.add(Conv2D(1024, (3,3), strides=(1,1), padding='same', name='conv_24'))
    model.add(LeakyReLU(alpha=0.1))
    
    model.add(Flatten())
    model.add(Dense(1024, activation='relu'))
    model.add(Dense(4096, activation='relu'))

    return model

[Arquitetura YOLO9000](https://arxiv.org/pdf/1612.08242.pdf)

De forma similar a primeira arquitetura da YOLO, a YOLO9000 possui seis blocos convolucionais principais, sendo utilizados filtros 3x3 para extração das caracteristicas e filtros 1x1 para redução de profundidade em cada camada. Além disso, a quantidade de filtros dobra a cada bloco, iniciando em 32 e obtendo 1024 na última camada, conforme Figura abaixo. 
![alt text](../imgs/yolov2_architecture.png "Title")

Entretanto, neste modelo, conhecido como Darknet-19, apenas 19 camadas convolucionais são utilizadas, sendo a última utilizada para identificar o objeto e suas coordenadas, eliminando-se assim a necessidade de utilizar camadas completamente conectadas e resultando em um modelo completamente convolucional. Essa abordagem facilita a predição em diferentes escalas e é também utilizada nos modelos [SSD](https://arxiv.org/pdf/1512.02325.pdf) (Single Shot Detector). Para isso, adiciona-se uma última camada convolucional de dimensões 1x1x1000, onde 1000 é a quantidade de classes existentes no dataset.

Além disso, cada camada convolucional é seguida de uma operação de [normalização de batch](https://arxiv.org/abs/1502.03167). Essa técnica é utilizada para normalizar a média e a variância de cada camada da rede, de forma que, ao utilizar uma arquitetura complexa como a YOLO9000 com 19 camadas convolucionais, a distribuição permaneça aproximadamente constante, tornando a rede assim mais estável. Para a YOLO, a utilização dessa abordagem melhora em 2% mAP e elimina a necessidade de utilizar Dropout.    

Por fim, essa arquitetura utiliza skip connections, um conceito abordado em [redes residuais](https://arxiv.org/abs/1512.03385), responsável por criar um bloco paralelo na rede. Com isso, uma skip connection é adicionada na camada 13 (última camada com 512 filtros) e depois concatenada com a última camada de 1024 filtros, de forma que a rede considere igualmente as características de resolução alta das últimas camadas e as de baixa resolução das primeiras camadas, melhorando a performance da rede em 1%. 

Essa última alteração é eficaz na YOLO devido a complexidade do modelo. Considerando que a arquitetura é muito densa, a medida que mais camadas convolucionais são adicionadas, o gradiente pode começar a ter problemas de convergência, aumentando-se assim a taxa de erro. Dessa forma, ao adicionar uma skip connection na metade do modelo (camada 13 de 24), o gradiente retém as informações extraídas das primeiras camadas e evita os problemas de convergência, ao mesmo tempo as características das camadas restantes são extraídas. 


In [54]:
def yolo2(input_image,num_classes):
    """
    Implementação da arquitetura da YOLO dividida nos seis blocos convolucionais e nas camadas
    completamente conectadas.
    Parâmetro: dimensão da imagem
    Retorna: modelo final implementado
    """   
        
    # Bloco 1 - Camadas de baixa profundidade (32 filtros)
    x = Conv2D(32, (3,3), strides=(1,1), padding='same', name='conv_1')(input_image)
    x = BatchNormalization(name='norm_1')(x)
    x = LeakyReLU(alpha=0.1)(x)
    x = MaxPooling2D(pool_size=(2, 2))(x)

    # Bloco 2 - Camadas de baixa profundidade (64 filtros)
    x = Conv2D(64, (3,3), strides=(1,1), padding='same', name='conv_2')(x)
    x = BatchNormalization(name='norm_2')(x)
    x = LeakyReLU(alpha=0.1)(x)
    x = MaxPooling2D(pool_size=(2, 2))(x)

    # Bloco 3 - Transição entre dimensões 64 e 128 a partir de camadas
    # convolucionais 1x1
    x = Conv2D(128, (3,3), strides=(1,1), padding='same', name='conv_3')(x)
    x = BatchNormalization(name='norm_3')(x)
    x = LeakyReLU(alpha=0.1)(x)
    
    x = Conv2D(64, (1,1), strides=(1,1), padding='same', name='conv_4')(x)
    x = BatchNormalization(name='norm_4')(x)
    x = LeakyReLU(alpha=0.1)(x)

    x = Conv2D(128, (3,3), strides=(1,1), padding='same', name='conv_5')(x)
    x = BatchNormalization(name='norm_5')(x)
    x = LeakyReLU(alpha=0.1)(x)
    x = MaxPooling2D(pool_size=(2, 2))(x)

    # Bloco 4 - Transição entre dimensões 128 e 256 a partir de camadas
    # convolucionais 1x1
    x = Conv2D(256, (3,3), strides=(1,1), padding='same', name='conv_6')(x)
    x = BatchNormalization(name='norm_6')(x)
    x = LeakyReLU(alpha=0.1)(x)

    x = Conv2D(128, (1,1), strides=(1,1), padding='same', name='conv_7')(x)
    x = BatchNormalization(name='norm_7')(x)
    x = LeakyReLU(alpha=0.1)(x)

    x = Conv2D(256, (3,3), strides=(1,1), padding='same', name='conv_8')(x)
    x = BatchNormalization(name='norm_8')(x)
    x = LeakyReLU(alpha=0.1)(x)
    x = MaxPooling2D(pool_size=(2, 2))(x)

    # Bloco 5 - Transição entre dimensões 512 e 256 a partir de camadas
    # convolucionais 1x1
    x = Conv2D(512, (3,3), strides=(1,1), padding='same', name='conv_9')(x)
    x = BatchNormalization(name='norm_9')(x)
    x = LeakyReLU(alpha=0.1)(x)

    x = Conv2D(256, (1,1), strides=(1,1), padding='same', name='conv_10')(x)
    x = BatchNormalization(name='norm_10')(x)
    x = LeakyReLU(alpha=0.1)(x)

    x = Conv2D(512, (3,3), strides=(1,1), padding='same', name='conv_11')(x)
    x = BatchNormalization(name='norm_11')(x)
    x = LeakyReLU(alpha=0.1)(x)
    
    x = Conv2D(256, (1,1), strides=(1,1), padding='same', name='conv_12')(x)
    x = BatchNormalization(name='norm_12')(x)
    x = LeakyReLU(alpha=0.1)(x)

    x = Conv2D(512, (3,3), strides=(1,1), padding='same', name='conv_13')(x)
    x = BatchNormalization(name='norm_13')(x)
    x = LeakyReLU(alpha=0.1)(x)
    
    #Adicionar skip connection para bloco residual
    skip_connection = x
    x = MaxPooling2D(pool_size=(2, 2))(x)

    # Bloco 6 - Transição entre dimensões 512 e 1024 a partir de camadas
    # convolucionais 1x1
    x = Conv2D(1024, (3,3), strides=(1,1), padding='same', name='conv_14', use_bias=False)(x)
    x = BatchNormalization(name='norm_14')(x)
    x = LeakyReLU(alpha=0.1)(x)

    x = Conv2D(512, (1,1), strides=(1,1), padding='same', name='conv_15', use_bias=False)(x)
    x = BatchNormalization(name='norm_15')(x)
    x = LeakyReLU(alpha=0.1)(x)
    
    x = Conv2D(1024, (3,3), strides=(1,1), padding='same', name='conv_16', use_bias=False)(x)
    x = BatchNormalization(name='norm_16')(x)
    x = LeakyReLU(alpha=0.1)(x)

    x = Conv2D(512, (1,1), strides=(1,1), padding='same', name='conv_17', use_bias=False)(x)
    x = BatchNormalization(name='norm_17')(x)
    x = LeakyReLU(alpha=0.1)(x)

    x = Conv2D(1024, (3,3), strides=(1,1), padding='same', name='conv_18', use_bias=False)(x)
    x = BatchNormalization(name='norm_18')(x)
    x = LeakyReLU(alpha=0.1)(x)

    #Bloco 7

    x = Conv2D(1024, (3,3), strides=(1,1), padding='same', name='conv_19', use_bias=False)(x)
    x = BatchNormalization(name='norm_19')(x)
    x = LeakyReLU(alpha=0.1)(x)

    x = Conv2D(1024, (3,3), strides=(1,1), padding='same', name='conv_20', use_bias=False)(x)
    x = BatchNormalization(name='norm_20')(x)
    x = LeakyReLU(alpha=0.1)(x)

    # Bloco Residual - concatenando modelo de rede de baixa resolução das primeiras 
    #camadas(até 512 filtros) e alta resolução das últimas camadas (1024 filtros)
    skip_connection = Conv2D(64, (1,1), strides=(1,1), padding='same', name='conv_21', use_bias=False)(skip_connection)
    skip_connection = BatchNormalization(name='norm_21')(skip_connection)
    skip_connection = LeakyReLU(alpha=0.1)(skip_connection)
    skip_connection = Lambda(space_to_depth_x2)(skip_connection)

    x = concatenate([skip_connection, x])
    
    x = Conv2D(1024, (3,3), strides=(1,1), padding='same', name='conv_22', use_bias=False)(x)
    x = BatchNormalization(name='norm_22')(x)
    x = LeakyReLU(alpha=0.1)(x)
    
    #Última camada para detecção, as dimensões de entrada para essa camada são obtidas
    #a partir da equação num_anchor_boxes * (num_classes+5).
    x = Conv2D((5*(num_classes+5)), (1,1), strides=(1,1), padding='same', name='conv_23', use_bias=False)(x)
    
    #Estruturando a saída para ter suporte ao sistema de grids da YOLO que divide a imagem em 
    #13x13 células.
    output = layers.Reshape((13, 13, 5, 4 + 1 + num_classes))(x)
         
    # small hack to allow true_boxes to be registered when Keras build the model 
    # for more information: https://github.com/fchollet/keras/issues/2790
    true_boxes  = layers.Input(shape=(1, 1, 1, 50, 4))
    output = layers.Lambda(lambda args: args[0])([output, true_boxes])
    
    model = models.Model([input_image, true_boxes], output)
    return model

In [50]:
### Testar modelos ###
yolo_model = yolo2(Input(shape=(416, 416, 3)),100)
#yolo_model.summary()

### Exportando modelo no Keras 

Os modelos implementados neste notebook foram feitos para ilustrar e melhorar o entendimento da arquitetura da YOLO, não sendo utilizados para detecção propriamente dita. 

Para a detecção, os pesos da rede treinada foram disponibilizados pelos autores da [YOLO](https://pjreddie.com/darknet/yolo/). Dessa forma, para finalizar esse notebook, o método load_model do Keras carrega o modelo da YOLO9000 e será utilizado nos demais notebooks que ilustrarão a implementação do algoritmo de detecção.

In [51]:
yolo_model = load_model("../model_data/yolo.h5")
yolo_model.summary()

__________________________________________________________________________________________________
Layer (type)                    Output Shape         Param #     Connected to                     
input_1 (InputLayer)            (None, 608, 608, 3)  0                                            
__________________________________________________________________________________________________
conv2d_1 (Conv2D)               (None, 608, 608, 32) 864         input_1[0][0]                    
__________________________________________________________________________________________________
batch_normalization_1 (BatchNor (None, 608, 608, 32) 128         conv2d_1[0][0]                   
__________________________________________________________________________________________________
leaky_re_lu_1 (LeakyReLU)       (None, 608, 608, 32) 0           batch_normalization_1[0][0]      
__________________________________________________________________________________________________
max_poolin

/usr/local/lib/python3.6/dist-packages/keras/models.py:252: UserWarning: No training configuration found in save file: the model was *not* compiled. Compile it manually.
  warnings.warn('No training configuration found in save file: '
